In [1]:
import pandas as pd

In [57]:
param_in_data_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"

## Read all the ~6000 fields chosen previously

In [58]:
evaluation_set = pd.read_csv(param_in_data_dir + "Eshwar_Extensive.csv")
evaluation_set.drop(labels=['Text box Text', 'Question_overall'], axis='columns', inplace=True)
print (evaluation_set.shape)
evaluation_set.head(1)

(6340, 14)


,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.832739,-118.912092,How would you label this field?,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams


In [59]:
# (sorted(evaluation_set.CropTyp.unique()))

In [60]:
wanted_crops = ['alfalfa seed',
                'barley', 'barley hay', 'bean, dry', 'bean, green', 'bluegrass seed', 'buckwheat', 
                'canola', 'carrot', 'corn seed', 'corn, field', 'corn, sweet', 
                'grass hay', 'grass seed',
                'market crops', 'mint',
                'oat hay', 'onion',
                'pea seed', 'pea, dry', 'pea, green', 'potato', 
                'triticale',
                'wheat',
                'yellow mustard']
wanted_crops = sorted(wanted_crops)
len(wanted_crops)

25

In [62]:
print (evaluation_set.shape)
evaluation_set = evaluation_set[evaluation_set.CropTyp.isin(wanted_crops)]
print (evaluation_set.shape)

(6340, 14)
(1925, 14)


## Read the first set; 294 fields labeled in the first set

In [63]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
set1_fields = pd.read_csv(perry_dir + "set_1_experts_stats_extended_sortOpinionCrop.csv")

In [64]:
hand_picked_set2 = pd.read_csv(param_in_data_dir + "threeHundred_IDs_Set2_Perry.csv")
hand_picked_set2.dropna(inplace=True)

hand_picked_set2 = evaluation_set[evaluation_set.ID.isin(list(hand_picked_set2.ID))].copy()
hand_picked_set2.reset_index(drop=True, inplace=True)
hand_picked_set2.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,148920_WSDA_SF_2015,Walla2015_148920_WSDA_SF_2015.png,148920_WSDA_SF_2015_46.03609783_-118.72012153_...,148920_WSDA_SF_2015_46.03609783_-118.72012153_...,46.036098,-118.720122,How would you label this field?,alfalfa seed,sprinkler,wsda,2,2.197327,2015/06/04 00:00:00,Walla Walla
1,152722_WSDA_SF_2015,Walla2015_152722_WSDA_SF_2015.png,152722_WSDA_SF_2015_46.03008259_-118.69196771_...,152722_WSDA_SF_2015_46.03008259_-118.69196771_...,46.030083,-118.691968,How would you label this field?,alfalfa seed,sprinkler,wsda,33,33.308379,2015/06/04 00:00:00,Walla Walla


In [65]:
hand_picked_set2.shape

(322, 14)

In [66]:
set1_fields.shape

(294, 21)

In [67]:
# sorted(set1_fields.CropTyp.unique())
[x for x in set1_fields.CropTyp.unique() if not(x in wanted_crops)]

['triticale hay', 'carrot seed', 'wheat fallow']

# Find out CropType of repeated fields!

In [79]:
repeated_crops = hand_picked_set2[hand_picked_set2.ID.isin(list(set1_fields.ID))].copy()
repeated_crops.CropTyp.unique()

array(['alfalfa seed', 'barley', 'barley hay', 'bean, dry', 'bean, green',
       'bluegrass seed', 'buckwheat', 'canola', 'carrot', 'corn seed',
       'corn, sweet', 'grass hay', 'grass seed', 'market crops', 'mint',
       'oat hay', 'onion', 'pea, dry', 'pea, green', 'triticale',
       'yellow mustard'], dtype=object)

In [92]:
repeated_crops.drop(labels=['NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB',
                            'latitude', 'longitude', 'Question Text',
                            'Irrigtn', 'DataSrc', 'Acres',
                            'ExctAcr', 'LstSrvD', 'county'], 
                         axis='columns', inplace=True)
print (repeated_crops.shape)

(141, 14)

In [111]:
repeated_crops.head(2)

,ID,CropTyp
0,148920_WSDA_SF_2015,alfalfa seed
1,152722_WSDA_SF_2015,alfalfa seed


# Toss out the first and second sets!!!!

In [86]:
set3_potentials = evaluation_set.copy()
print (set3_potentials.shape)
set3_potentials = set3_potentials[set3_potentials.CropTyp.isin(list(repeated_crops.CropTyp.unique()))]
set3_potentials = set3_potentials[~set3_potentials.ID.isin(list(set1_fields.ID))].copy()
print (set3_potentials.shape)

set3_potentials = set3_potentials[~set3_potentials.ID.isin(list(hand_picked_set2.ID))].copy()

print (set3_potentials.shape)

set3_potentials.sort_values(by=['CropTyp', 'county', "ID"], inplace=True)

(1925, 14)
(786, 14)
(631, 14)


In [87]:
set3_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set3/"
out_name = set3_dir + "00_set3_all_potentials.csv"
set3_potentials.to_csv(out_name, index = False)

In [88]:
set3_potentials.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
495,160565_WSDA_SF_2015,Walla2015_160565_WSDA_SF_2015.png,160565_WSDA_SF_2015_46.02407437_-118.62400002_...,160565_WSDA_SF_2015_46.02407437_-118.62400002_...,46.024074,-118.624000,How would you label this field?,alfalfa seed,center pivot,wsda,54,53.999581,2015/06/16 00:00:00,Walla Walla
497,160915_WSDA_SF_2015,Walla2015_160915_WSDA_SF_2015.png,160915_WSDA_SF_2015_46.01757824_-118.59877151_...,160915_WSDA_SF_2015_46.01757824_-118.59877151_...,46.017578,-118.598772,How would you label this field?,alfalfa seed,sprinkler,wsda,21,20.672131,2015/06/16 00:00:00,Walla Walla


# Manually Picked

I manually picked some fields from ```00_set3_all_potentials```.
There were not enough of ```oat hay``` and ```yellow mustard```.
So, I replaced them with ```onion``` and ```pea, green```

In [124]:
manual_picks = pd.read_csv(set3_dir + "01_Manually_Picked_IDs.csv")
manual_picks.drop(labels=['row', 'Crop'], axis='columns', inplace=True)

In [125]:
print (manual_picks.shape)
print (len(manual_picks.ID.unique()))
manual_picks.head(2)

(142, 1)
142


,ID
0,161245_WSDA_SF_2015
1,169545_WSDA_SF_2015


In [126]:
manual_picks = pd.merge(manual_picks, set3_potentials, on=['ID'], how='left')
manual_picks.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,161245_WSDA_SF_2015,Walla2015_161245_WSDA_SF_2015.png,161245_WSDA_SF_2015_46.02743904_-118.6990808_c...,161245_WSDA_SF_2015_46.02743904_-118.6990808_T...,46.027439,-118.699081,How would you label this field?,alfalfa seed,sprinkler,wsda,10,10.401307,2015/06/04 00:00:00,Walla Walla
1,169545_WSDA_SF_2015,Walla2015_169545_WSDA_SF_2015.png,169545_WSDA_SF_2015_46.016413_-118.54690439_co...,169545_WSDA_SF_2015_46.016413_-118.54690439_TO...,46.016413,-118.546904,How would you label this field?,alfalfa seed,sprinkler,wsda,80,79.567760,2015/06/17 00:00:00,Walla Walla


In [127]:
# manual_picks.groupby(['CropTyp']).count()
# repeated_crops.groupby(['CropTyp']).count()

In [128]:
out_name = set3_dir + "manual_picks_full_columns.csv"
manual_picks.to_csv(out_name, index = False)

In [139]:
no_questions = 50

writer_limited = pd.ExcelWriter(set3_dir + 'set_3_handPicked.xlsx', engine='xlsxwriter') 

if manual_picks.shape[0] % no_questions != 0:
    no_dfs = manual_picks.shape[0] // no_questions + 1
else:
    no_dfs = manual_picks.shape[0] // no_questions

for ii in range(no_dfs):
    curr_sheet = manual_picks.loc[(ii*no_questions): ((ii+1) * no_questions) - 1, ]
    curr_sheet.reset_index(drop=True, inplace=True)
    curr_sheet.to_excel(writer_limited, sheet_name= "set_3_form_" + str(ii+1), index=False)

writer_limited.save()

In [137]:
manual_picks.head(2)

,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,161245_WSDA_SF_2015,Walla2015_161245_WSDA_SF_2015.png,161245_WSDA_SF_2015_46.02743904_-118.6990808_c...,161245_WSDA_SF_2015_46.02743904_-118.6990808_T...,46.027439,-118.699081,How would you label this field?,alfalfa seed,sprinkler,wsda,10,10.401307,2015/06/04 00:00:00,Walla Walla
1,169545_WSDA_SF_2015,Walla2015_169545_WSDA_SF_2015.png,169545_WSDA_SF_2015_46.016413_-118.54690439_co...,169545_WSDA_SF_2015_46.016413_-118.54690439_TO...,46.016413,-118.546904,How would you label this field?,alfalfa seed,sprinkler,wsda,80,79.567760,2015/06/17 00:00:00,Walla Walla


In [138]:
manual_picks.columns

Index(['ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude',
       'longitude', 'Question Text', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres',
       'ExctAcr', 'LstSrvD', 'county'],
      dtype='object')

In [135]:
evaluation_set.columns

Index(['ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude',
       'longitude', 'Question Text', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres',
       'ExctAcr', 'LstSrvD', 'county'],
      dtype='object')

# Check the google sheet 
```File List - Perry and Co.``` renamed to ```FileList-PerryandCo```

In [141]:
FileList = pd.ExcelFile(set3_dir + "FileList-PerryandCo.xlsx")
FileList_sheet_names = FileList.sheet_names  # see all sheet names
print (FileList_sheet_names)

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6', 'set_2_form_1', 'set_2_form_2', 'set_2_form_3', 'set_2_form_4', 'set_2_form_5', 'set_2_form_6', 'set_2_form_7', 'set_3_form_1', 'set_3_form_2', 'set_3_form_3']


In [146]:
extended = pd.DataFrame()
set_2 = pd.DataFrame()
set_3 = pd.DataFrame()

for a_sheet in FileList_sheet_names:
    # read a damn sheet
    sheetdata = FileList.parse(a_sheet)
    
    if "set_3" in a_sheet:
        set_3 = pd.concat([set_3, sheetdata])
    elif "set_2" in a_sheet:
        set_2 = pd.concat([set_2, sheetdata])
    elif "extended" in a_sheet:
        extended = pd.concat([extended, sheetdata])

In [159]:
A = set_3[set_3.ID.isin(list(set_2.ID))]
A.shape

(0, 14)